Instructions for demo creators: this template is a rough draft to create UDT demo notebooks. 
You will need to fill in each block with information specific to your problem type (any snake
case variable names should be replaced with the actual values).

# problem_name with ThirdAI's Universal Deep Transformer

This notebook shows how to build a problem_name model with ThirdAI's
Universal Deep Transformer (UDT) model, our all-purpose classifier for tabular datasets.
In this demo, we will train and evaluate the model on the dataset_name dataset, 
but you can easily replace this with your own dataset.

To run this notebook, you will need to obtain a ThirdAI license at the following link if you have not already:
https://www.thirdai.com/try-bolt/



In [ ]:
!pip3 install thirdai
!pip3 install pandas # We need Pandas in the utils module to convert downloaded datasets to CSV format
!pip3 install numpy # We use numpy to analyze UDT performance in this notebook

# Dataset Download

We will use the utils module in this repo to download dataset_name (if you have just copied this notebook and not cloned the entire repo, you will need to copy the utils.py file as well). You can replace 
this step and the next step with a download method and a UDT initilization step
that is specific to your dataset.

In [ ]:
import utils
train_filename, test_filename, inference_batch = utils.download_dataset_name()

# UDT Initilization

We can now create a UDT model by passing in the types of each column in the dataset
and the target column we want to be able to predict.


Instructions to demo creators: if this is a time series UDT problem, include the following as well:

For this demo, we additionally want to use "temporal context" to make predictions.
Adding temporal context requires a single bolt.types.date() column to use to
track the timestamp of training data. We pass in a dictionary called 
temporal_tracking_relationships that tells UDT we want to track value_variable_name
over time for each key_variable_name. This allows UDT to make better predictions for
the target column by creating temporal features that take into account the 
historical relationship between key_variable_name and value_variable_name. 

We customize the "lookahead" and "time_granularity" for this temporal UDT model:
the model will then predict insert_lookahead_amount number of insert_time_granularity_value into the 
future. The model will use "insert_time_granularity_value" as the bin size
to group temporal numerical features during training and inference.

In [ ]:
from thirdai import bolt
model = bolt.UniversalDeepTransformer(
    data_types={
        "text": bolt.types.text(),
        "timestamp": bolt.types.date(),
        "numerical": bolt.types.numerical(range=(0, 25)),
        "category_1": bolt.types.categorical(n_unique_classes=10),
        "category_2": bolt.types.categorical(n_unique_classes=10)
    },
    target="category_2",
    # Remove below if not a temporal demo, otherwise fill in with correct values
    # temporal_tracking_relationships={"category_1": ["numerical"]},
    # lookahead=5,
    # time_granularity="weekly"
)

# Training

We can now train our UDT model with just two lines! Feel free to customize the
number of epochs and the learning rate; we have chosen values that give good
convergence.

In [ ]:
train_config = (bolt.TrainConfig(epochs=5, learning_rate=0.01)
                    .with_metrics(["categorical_accuracy"]))

model.train(train_filename, train_config)

# Evaluation

Evaluating the performance of the UDT model is also just two lines! 

In [ ]:
eval_config = bolt.EvalConfig().with_metrics(["categorical_accuracy"])

model.evaluate(test_filename, eval_config)

# Saving and Loading

Saving and loading a trained UDT model to disk is also extremely straight forward.

In [ ]:
save_location = "problem_name.model"

# Saving
model.save(save_location)

# Loading
model = bolt.UniversalDeepTransformer.load(save_location)


# Testing Predictions

The evaluation method is great for testing, but it requires labels, which don't
exist in a production setting. We also have a predict method that can take in an 
in-memory batch of rows or a single row (without the target column), allowing 
easy integration into production pipelines.

In [ ]:
import numpy as np

print("Inference batch:", inference_batch)

prediction = model.predict(inference_batch[0])
prediction_batch = model.predict_batch(inference_batch)

print("Inference prediction:", model.class_name(np.argmax(prediction)))